In [23]:
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [24]:
df = pd.read_csv("../Data/citibike_2022_with_weather_sample.csv", low_memory=False)
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp
0,C31A06A81423A14C,classic_bike,2022-01-18 07:40:25.346,2022-01-18 07:43:35.441,W 26 St & 8 Ave,6297.02,10 Ave & W 28 St,6459.04,40.747348,-73.997236,40.750664,-74.001768,member,2022-01-18,1.8
1,6104B30E6CC08F21,classic_bike,2022-01-18 20:13:23.444,2022-01-18 20:57:37.652,Jay St & York St,4895.09,Clark St & Henry St,4789.03,40.701403,-73.986727,40.697601,-73.993446,member,2022-01-18,1.8
2,127F7DF7F787F370,classic_bike,2022-01-01 02:26:22.776,2022-01-01 02:46:12.503,Greenwich St & Perry St,5922.04,E 13 St & Avenue A,5779.09,40.734980,-74.006970,40.729667,-73.980680,casual,2022-01-01,11.6
3,FF12FDCC260D47C3,classic_bike,2022-01-06 18:30:28.542,2022-01-06 18:44:35.763,Lexington Ave & E 36 St,6313.10,W 26 St & 10 Ave,6382.05,40.747574,-73.978801,40.749718,-74.002950,member,2022-01-06,4.9
4,D8402474AF50D5F1,classic_bike,2022-01-27 18:29:15.544,2022-01-27 18:30:52.077,31 St & 21 Ave,7202.07,31 St & Ditmars Blvd,7144.02,40.778130,-73.908420,40.776168,-73.910485,member,2022-01-27,-4.2


In [29]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")

df_2022 = df[
    (df["date"] >= "2022-01-01") &
    (df["date"] <  "2023-01-01")
].copy()

print(df_2022.shape)



(3524490, 15)


In [32]:
df_small = df.sample(n=5000, random_state=42)
df_small.to_csv("../Data/citibike_2022_small.csv", index=False)
print(df_small.shape)


(5000, 15)


In [33]:
df_small["trip_count"] = 1

df_final = df_small.groupby(
    [
        "start_station_name",
        "end_station_name",
        "start_lat",
        "start_lng",
        "end_lat",
        "end_lng"
    ],
    as_index=False
).agg({"trip_count": "sum"})


In [ ]:
# remove date column if exists
df_small = df_small.drop(columns=["date"], errors="ignore")
df_final = df_final.drop(columns=["date"], errors="ignore")


In [40]:
from keplergl import KeplerGl

m = KeplerGl(height=700)
m.add_data(df_small, "Bike Points")
m.add_data(df_final, "Bike Arcs")

m.save_to_html(file_name="../Docs/citibike_small_map.html")


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\mais1\anaconda3\envs\kepler310\lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Map saved to ../Docs/citibike_small_map.html!


c:\Users\mais1\anaconda3\envs\kepler310\lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [41]:
import json

with open("kepler_config.json", "r") as f:
    config = json.load(f)

m = KeplerGl(height=700, config=config)
m.add_data(df_small, "Bike Points")
m.add_data(df_final, "Bike Arcs")


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\mais1\anaconda3\envs\kepler310\lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
c:\Users\mais1\anaconda3\envs\kepler310\lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


## Kepler Map 

![Citibike Routes](../Imgs/citibike_routes_small.png)

I changed the Point layer color to make individual stations more visible and to highlight high-density areas.
I added an Arc layer to show movement between stations, which helps visualize flow direction and traffic intensity.
Colors were adjusted for contrast so that routes (arcs) stand out from the station points.
These changes make the busiest regions and trip patterns easier to see at a glance.

In [51]:
import json
from keplergl import KeplerGl

# load config from file
with open(r"D:\CareerFoundary\CitiBike_2022_Project\kepler_config.json", "r") as f:
    config = json.load(f)

# prepare data dictionary for Kepler
data = {
    "Bike Points": df_small,
    "Bike Arcs": df_final
}

# create KeplerGl map with data and config
m = KeplerGl(height=700, data=data, config=config)

# save map to HTML file
m.save_to_html(file_name="../Docs/citibike_map_config_test.html")


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\mais1\anaconda3\envs\kepler310\lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Map saved to ../Docs/citibike_map_config_test.html!


## Temperature Filter Analysis (15–25 °C)

![Temper_Filter ](../Imgs/citibike_routes_small_filter.png)

A filter was applied to explore how bike usage changes with weather conditions.
By isolating trips within an ideal temperature range of ≈15–25°C, we observe visibly denser activity, especially across central Manhattan.